### Working with documents in Python.

In [1]:
%pip install langchain
%pip install langchain-ollama 
%pip install langchain-community
%pip install unstructured
%pip install pdfminer
%pip install pi-heif
%pip install -qU langchain-chroma
#%pip install chromadb
%pip uninstall -y pdfminer.six
%pip install pdfminer.six>=20231228
%pip install --upgrade unstructured[pdf]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Found existing installation: pdfminer.six 20251107
Uninstalling pdfminer.six-20251107:
  Successfully uninstalled pdfminer.six-20251107
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


The above code worked on virtual environment Python 3.1.3.10. So what I have done is I have downgraded my current installed version to the current installed version of the lower environment. Downgraded from 3.14 to 3.13 and now we are able to run the code. This took a really long time but we are able to learn it. Thank you 

In [ ]:
# import os
# from langchain_community.document_loaders import UnstructuredPDFLoader

# # 1. Create the empty list to store all document chunks
# documents = []

# # 2. Define the directory where your PDFs are stored
# # Replace './your_pdf_folder' with the actual path to your folder
# directory = './Docs'

# # 3. Loop through files in the directory
# for filename in os.listdir(directory):
    
#     # Check if the file is actually a PDF
#     if filename.endswith('.pdf'):
        
#         # 4. Construct the full file path (os.path.join)
#         # The transcript emphasized this was crucial to avoid errors
#         file_path = os.path.join(directory, filename)
        
#         # 5. Create the loader with the file path
#         loader = UnstructuredPDFLoader(file_path)
        
#         # 6. Load the file and append (extend) it to the main documents list
#         documents.extend(loader.load())

# # Verify the data was loaded
# print(f"Successfully loaded {len(documents)} documents.")

Cannot set gray non-stroke color because /'P40' is an invalid float value


Successfully loaded 3 documents.


In [1]:
import os
from langchain_community.document_loaders import (
UnstructuredPDFLoader
) 
doc_folder = './Docs'
documents = []   
for filename in os.listdir(doc_folder):
        file_path = os.path.join(doc_folder, filename)
        loader = UnstructuredPDFLoader(file_path)
        documents.extend(loader.load())

e:\LocalFlow\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cannot set gray non-stroke color because /'P40' is an invalid float value


### Splitting documents into smaller chunks 

The smaller chunks are the better. First time when I was trying to do chunking, I kept the chunk size as 10,000 but it did not work. In the next step it failed. Then I realized the importance of chunk size is important and it gives the break between the tasks for the CPU so it would be more easy.

What I have done is now we have created a chunk smaller chunks with the size of 1,000 and we are able to proceed further 

In [7]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_documents(documents)
print(f"Number of chunks created: {len(chunks)}")   
for i, chunk in enumerate(chunks[:3]):  # Display first 3 chunks
    print(f"\n--- Chunk {i+1} ---\n")
    print(chunk.page_content)

Number of chunks created: 2155

--- Chunk 1 ---

FFIRS 08/25/2011 11:31:15 Page 2

FFIRS 08/25/2011 11:31:15 Page 1

THE ART OF SOFTWARE TESTING

FFIRS 08/25/2011 11:31:15 Page 2

FFIRS 08/25/2011 11:31:15 Page 3

THE ART OF SOFTWARE TESTING

Third Edition

GLENFORD J. MYERS TOM BADGETT COREY SANDLER

John Wiley & Sons, Inc.

FFIRS 08/25/2011 11:31:15 Page 4

Copyright # 2012 by Word Association, Inc. All rights reserved.

Published by John Wiley & Sons, Inc., Hoboken, New Jersey.

Published simultaneously in Canada.

--- Chunk 2 ---

FFIRS 08/25/2011 11:31:15 Page 4

Copyright # 2012 by Word Association, Inc. All rights reserved.

Published by John Wiley & Sons, Inc., Hoboken, New Jersey.

Published simultaneously in Canada.

No part of this publication may be reproduced, stored in a retrieval system, or transmitted in any form or by any means, electronic, mechanical, photocopying, recording, scanning, or otherwise, except as permitted under Section 107 or 108 of the 1976 United State

Embedding Documents 

In [8]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
##embeddings = OllamaEmbeddings(model="embeddings-gemma")
embeddings = OllamaEmbeddings(model="nomic-embed-text")
db = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db")

retriver = db.as_retriever()

In [9]:
embeddings

OllamaEmbeddings(model='nomic-embed-text', validate_model_on_init=False, base_url=None, client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [10]:
retriever = db.as_retriever()


In [11]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="gemma3:1b",
    temperature=0.7
)

### retrieval QA 

Result = qa_chain.invoke(query). We can also use invoke instead of invoke. We can also use run 

In [14]:
from langchain_classic.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)
query = "Summarize the key points discussed in the documents regarding local development environments."
result = qa_chain.invoke(query)
print("Answer:")
print(result['result']) 

Answer:
Okay, here’s a summary of the key points regarding local development environments, based on the provided text:

**Key Points on Local Development Environments:**

*   **Definition:** Local development environments are environments where software development takes place, allowing for more control, security, and privacy.
*   **Importance:** They’re crucial for agile development methodologies, particularly the Agile Manifesto, as they emphasize customer-centricity and iterative development.
*   **Core Principles:**
    *   **Customer-Focused:** Development revolves around meeting customer needs and expectations.
    *   **Iterative & Incremental:** Development happens in small, manageable steps.
    *   **Change-Driven:** The development process embraces changes and adapts to evolving requirements.
*   **Stages of Development:** The text outlines a typical software development lifecycle (SDLC) that includes:
    1.  **Requirements Translation:** Clearly defining the goals and requ